# LESO - Image preparation for the MPS process

## Cropping Images
Cortar las imágenes según la máscara para aislar el canal y eliminar todo el resto. La imagen se gira para enderezar el canal, luego se  corta para dejar el canal al centro y finalmente se convierte a negro todo lo que está por fuera del canal.

In [ ]:
import os
from PIL import Image, ImageChops
import matplotlib.pyplot as plt

Carpetas donde están las imágenes originales, dónde guardarlas y dónde está guardada la máscara.

In [ ]:
# Folder containing the JPG images and the mask
folder_path = r"H:\Experiment20_20240523" # Where RAW images are stored
output_folder_path = r"E:\Exp20\cropped"
mask_path = r"D:\LESO\DATA_RAW"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [ ]:
mask_filename = "mask.png"

# List all JPG files in the folder
jpg_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]

# Load the mask
mask_path = os.path.join(mask_path, mask_filename)
mask = Image.open(mask_path).convert("L")  # Convert to grayscale
mask = mask.rotate(3.6, expand=True, fillcolor="black")

Todas las imágenes de la carpeta de origen son cortadas y rotadas.

In [ ]:
# Loop through each JPG file
for jpg_file in jpg_files:
    # Construct the full file path for the input image
    input_image_path = os.path.join(folder_path, jpg_file)

    # Open the input image using Pillow
    input_image = Image.open(input_image_path)
    input_image = input_image.rotate(-90, expand=True, fillcolor="black")
    
    # Rotate the input image by 3.6 degrees counterclockwise
    rotated_image = input_image.rotate(3.6, expand=True, fillcolor="black")
    
    # Apply the mask to keep only the region of interest
    result_image = ImageChops.multiply(
        rotated_image.convert("RGB"), mask.convert("RGB")
    )

    # Find the bounding box of the non-black region in the inverted image
    bbox = result_image.getbbox()
    result_image = result_image.crop(bbox)

    # Construct the file path for the cropped image in the output folder
    output_image_path = os.path.join(output_folder_path, "cropped_" + jpg_file)

    # Save the processed image in the output folder
    result_image.save(output_image_path)

    # Close the image files
    input_image.close()
    rotated_image.close()
    result_image.close()
    
print("Image processing completed.")

## Binary Image Creation - Color Thresholding 
Filtering using blue ink as indicator of water surface. The HSV space is the most suitable for this type of thresholding. MATLAB provides the theshold values with its Image Thresholding Toolbox.

In [ ]:
import cv2
import numpy as np
import os

# Convert MATLAB range to OpenCV range
def convert_to_opencv_range(matlab_value, opencv_max):
    return int(matlab_value * opencv_max)

# Function to process an image and create a mask
def process_image(image_path):
    # Read the image
    img = cv2.imread(image_path)
    
    # Convert the image to HSV
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Create a mask using the inRange function
    mask = cv2.inRange(hsv_img, lower_hsv, upper_hsv)
    
    return mask

In [ ]:
# All the water surface thresholds.
# Define thresholds for channel 1 based on MATLAB settings
# Define thresholds for channel 1 based on histogram settings
channel1Min = 0.401;
channel1Max = 0.662;

# Define thresholds for channel 2 based on histogram settings
channel2Min = 0.180;
channel2Max = 1.000;

# Define thresholds for channel 3 based on histogram settings
channel3Min = 0.049;
channel3Max = 1.000;


# Convert MATLAB range to OpenCV range
channel1Min_opencv = convert_to_opencv_range(channel1Min, 179)
channel1Max_opencv = convert_to_opencv_range(channel1Max, 179)
channel2Min_opencv = convert_to_opencv_range(channel2Min, 255)
channel2Max_opencv = convert_to_opencv_range(channel2Max, 255)
channel3Min_opencv = convert_to_opencv_range(channel3Min, 255)
channel3Max_opencv = convert_to_opencv_range(channel3Max, 255)

# channel1Min_opencv = 68
# channel1Max_opencv = 109
# channel2Min_opencv = 120
# channel2Max_opencv = 255
# channel3Min_opencv = 0
# channel3Max_opencv = 255



# Define HSV filter range
lower_hsv = np.array([channel1Min_opencv, channel2Min_opencv, channel3Min_opencv])
upper_hsv = np.array([channel1Max_opencv, channel2Max_opencv, channel3Max_opencv])

# Process all JPG files in a folder
folder_path = r"E:\Exp20\cropped"
output_folder = r"E:\Exp20\masked\all"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        # Create the full path to the input and output files
        input_path = os.path.join(folder_path, filename)
        output_path = os.path.join(output_folder, "masked_"+filename[8:])

        # Process the image and save the result
        mask = process_image(input_path)
        cv2.imwrite(output_path, mask)
        
print("Masking process finished.")

## Mask resize and other operations

### Only resizing

In [ ]:
import cv2
import os

# Function to resize an image while maintaining aspect ratio
def resize_image(input_path, output_path, target_size):
    # Read the image
    img = cv2.imread(input_path)
    
    # Get the original image dimensions
    height, width = img.shape[:2]

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Calculate the new dimensions based on the target size and aspect ratio
    if aspect_ratio > 1:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_width = int(target_size * aspect_ratio)
        new_height = target_size

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height),interpolation = cv2.INTER_NEAREST)
    

    # Save the resized image
    cv2.imwrite(output_path, resized_img)

In [ ]:
# Process all JPG files in the input folder
input_folder = r"E:\Exp20\masked\deepest" #where the original-size masked images are stored.
output_folder = r"E:\Exp20\resized\deepest" #where to store the resized images
target_size = 200  # Set your desired target size

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        # Create the full path to the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, "resized_"+filename[7:-4]+".png")
        # Resize the image and save the result
        resize_image(input_path, output_path, target_size)

print("Resizing process finished.")

### Mask resize and inversion of color

In [ ]:
import cv2
import os

# Function to resize an image while maintaining aspect ratio and invert colors
def resize_and_invert_image(input_path, output_path, target_size):
    # Read the image
    img = cv2.imread(input_path)
    
    # Get the original image dimensions
    height, width = img.shape[:2]

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Calculate the new dimensions based on the target size and aspect ratio
    if aspect_ratio > 1:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_width = int(target_size * aspect_ratio)
        new_height = target_size

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
    
    # Invert the colors
    inverted_img = 255 - resized_img

    # Save the inverted image
    cv2.imwrite(output_path, inverted_img)


In [ ]:
# Process all JPG files in the input folder
input_folder = r"E:\Exp20\masked\deepest" #where the original-size masked images are stored.
output_folder = r"E:\Exp20\resized\deepest-inversed" #where to store the resized images
target_size = 200  # Set your desired target sizee

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        # Create the full path to the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, "resized_"+filename[7:-4]+".png")
        # Resize the image and save the result
        resize_and_invert_image(input_path, output_path, target_size)

print("Resizing process finished.")

### Mask resize and inversion of color and edging

In [ ]:
import cv2
import os

# Function to resize an image, invert colors, and extract edges of black areas
def resize_invert_and_edge_image(input_path, output_path, target_size):
    # Read the image
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)  # Ensure the image is read in grayscale mode
    
    # Get the original image dimensions
    height, width = img.shape[:2]

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Calculate the new dimensions based on the target size and aspect ratio
    if aspect_ratio > 1:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_width = int(target_size * aspect_ratio)
        new_height = target_size

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
    
    # Invert the colors
    inverted_img = 255 - resized_img

    # Apply Canny edge detection
    # Note: You may need to adjust the threshold values based on your specific images
    edges = cv2.Canny(inverted_img, 0, 200)

    # Save the image with edges
    cv2.imwrite(output_path, edges)


#### HSV1 

In [ ]:
# Process all JPG files in the input folder
input_folder = r"E:\Exp20\masked\deepest" #where the original-size masked images are stored.
output_folder = r"E:\Exp20\resized\deepest-03resized_edged" #where to store the resized images
target_size = 200  # Set your desired target size

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        # Create the full path to the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, "resized_"+filename[7:-4]+".png")
        # Resize the image and save the result
        resize_invert_and_edge_image(input_path, output_path, target_size)

print("Resizing process finished.")

### Mask resize and inversion of color and edging and biggest blob

In [ ]:
import cv2
import numpy as np

def resize_keep_largest_blob_invert_and_edge_image(input_path, output_path, target_size):
    # Read the image in grayscale mode
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    
    # Get the original image dimensions
    height, width = img.shape[:2]

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Calculate the new dimensions based on the target size and aspect ratio
    if aspect_ratio > 1:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_width = int(target_size * aspect_ratio)
        new_height = target_size

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_NEAREST)

    # Find all blobs
    num_labels, labels_im = cv2.connectedComponents(resized_img)

    # Keep only the largest blob (excluding the background blob at index 0)
    if num_labels > 1:  # Ensure there is at least one blob plus background
        blob_sizes = np.bincount(labels_im.flatten())[1:]  # Count sizes of each blob, excluding background
        largest_blob_idx = np.argmax(blob_sizes) + 1  # Find the index of the largest blob (+1 to account for background being index 0)
        largest_blob_mask = (labels_im == largest_blob_idx).astype(np.uint8) * 255

        # Use the mask of the largest blob for further processing
        processed_img = largest_blob_mask
    else:
        # If no blobs were found, proceed with the original image
        processed_img = resized_img

    # Invert the colors
    inverted_img = 255 - processed_img

    # Apply Canny edge detection
    edges = cv2.Canny(inverted_img, 100, 200)

    # Save the image with edges
    cv2.imwrite(output_path, edges)


#### HSV1 

In [ ]:
# Process all JPG files in the input folder
input_folder = r"E:\Exp20\masked\deepest" #where the original-size masked images are stored.
output_folder = r"E:\Exp20\resized\deepest-03resized_edged_bigblob" #where to store the resized images

target_size = 200  # Set your desired target size

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        # Create the full path to the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, "resized_"+filename[7:-4]+".png")
        # Resize the image and save the result
        resize_keep_largest_blob_invert_and_edge_image(input_path, output_path, target_size)

print("Resizing process finished.")

### Mask resize and inversion of color and edging after removing smallest blobs

In [ ]:
import cv2
import numpy as np

def resize_remove_small_blobs_invert_and_edge_image(input_path, output_path, target_size, size_threshold):
    # Read the image in grayscale mode
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    
    # Get the original image dimensions
    height, width = img.shape[:2]

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Calculate the new dimensions based on the target size and aspect ratio
    if aspect_ratio > 1:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_width = int(target_size * aspect_ratio)
        new_height = target_size

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_NEAREST)

    # Find all blobs and their stats
    num_labels, labels_im, stats, _ = cv2.connectedComponentsWithStats(resized_img)

    # Create an output image initialized to all zeros (black)
    output_img = np.zeros_like(resized_img)

    # Go through all blobs (excluding the background blob at index 0)
    for i in range(1, num_labels):  # Start from 1 to skip background
        if stats[i, cv2.CC_STAT_AREA] >= size_threshold:
            # Add blob to output image if its size is above the threshold
            output_img[labels_im == i] = 255

    # Invert the colors
    inverted_img = 255 - output_img

    # Apply Canny edge detection
    edges = cv2.Canny(inverted_img, 100, 200)

    # Save the image with edges
    cv2.imwrite(output_path, edges)


#### HSV1 

In [ ]:
# Process all JPG files in the input folder

input_folder = r"D:\LESO\DATA_PROCESSED\MPS\Images\Exp13\HSV3_deepest\02masked" #where the original-size masked images are stored.
output_folder = r"D:\LESO\DATA_PROCESSED\MPS\Images\Exp13\HSV3_deepest\03resized-newcode" #where to store the resized images

target_size = 200  # Set your desired target size

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        # Create the full path to the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, "resized_"+filename[7:-4]+".png")
        # Resize the image and save the result
        resize_remove_small_blobs_invert_and_edge_image(input_path, output_path, target_size, 20)

print("Resizing process finished.")

Move every other file to a new folder to reduce computation time.

In [ ]:
import os
import shutil

def copy_every_other_file(source_folder, destination_folder):
    # Create destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # List all files in the source folder
    files = sorted(os.listdir(source_folder))

    # Copy every other file
    for index, file_name in enumerate(files):
        if index % 2 == 0:  # Change this to index % 2 != 0 if you want to copy the other set of files
            source_file = os.path.join(source_folder, file_name)
            destination_file = os.path.join(destination_folder, file_name)
            if os.path.isfile(source_file):
                shutil.copy(source_file, destination_file)
                print(f"Copied: {file_name}")

# Usage example
source_folder = "D:\LESO\DATA_PROCESSED\MPS\Images\Exp13\HSV3_deepest\\03resized-newcode"
destination_folder = "D:\LESO\DATA_PROCESSED\MPS\Images\Exp13\HSV3_deepest\\03resized-newcode-selection"
copy_every_other_file(source_folder, destination_folder)


### Mask resize after removing smallest blobs

In [ ]:
import cv2
import numpy as np
import os

def resize_remove_small_blobs(input_path, output_path, target_size, size_threshold):
    # Read the image in grayscale mode
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    
    # Get the original image dimensions
    height, width = img.shape[:2]

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Calculate the new dimensions based on the target size and aspect ratio
    if aspect_ratio > 1:
        new_width = target_size
        new_height = int(target_size / aspect_ratio)
    else:
        new_width = int(target_size * aspect_ratio)
        new_height = target_size

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_NEAREST)

    # Find all blobs and their stats
    num_labels, labels_im, stats, _ = cv2.connectedComponentsWithStats(resized_img)

    # Create an output image initialized to all zeros (black)
    output_img = np.zeros_like(resized_img)

    # Go through all blobs (excluding the background blob at index 0)
    for i in range(1, num_labels):  # Start from 1 to skip background
        if stats[i, cv2.CC_STAT_AREA] >= size_threshold:
            # Add blob to output image if its size is above the threshold
            output_img[labels_im == i] = 255

    # Save the image with edges
    cv2.imwrite(output_path, output_img)


In [ ]:
# Process all JPG files in the input folde
input_folder = r"D:\LESO\DATA_PROCESSED\MPS\Images\Exp13\HSV3_deepest\02masked" #where the original-size masked images are stored.
output_folder = r"D:\LESO\DATA_PROCESSED\MPS\Images\Exp13\HSV3_deepest\03resized-newcode" #where to store the resized images

#output_folder = r"D:\LESO\DATA_RAW\Experiment13_20240301\deepest-03resized_removedBlobs-inversed"
target_size = 200  # Set your desired target size
size_threshold = 20

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        # Create the full path to the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, "resized_"+filename[7:-4]+".png")
        # Resize the image and save the result
        resize_remove_small_blobs(input_path, output_path, target_size, size_threshold)

print("Resizing process finished.")